In [ ]:
pip install -q yahoo_fin

In [2]:
!mkdir -p data results logs


In [3]:
from google.colab import drive
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [ ]:
pip install  git+https://github.com/ahsank/runml@testv2.01#egg=runml

In [5]:
import runml.pipeline
from runml import pipeline,findata


In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAllocHL(df, stop_loss, profit_factor=1):
    #df['Gain_f'] = (2*df['Gain']+df['Gain_l']+df['Gain_h'])/4
    df['Gain_f'] = df[['Gain','Gain_l']].max(axis=1)
    df['Gain_f'] = df[['Gain_h', 'Gain_f']].min(axis=1)
    df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain_f'])
    df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
    return df

In [ ]:
findata.EPOCHS=200
new = []
model = "gstock-12b-test"
tickers = [ 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']


In [ ]:
findata.EPOCHS=200
model = "val-12b-test"
new = ['DXC', 'FSLR', 'STLD', 'FICO', 'SRE', 'BKR', 'CTAS', 'PEP', 'ADI', 'ROP' ]
tickers = ['AAL', 'ADI', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL', 'PXD',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TCN', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

In [7]:
findata.EPOCHS=200
model = "cry-1a-test"
tickers = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'SOL-USD', 'TRX-USD', 'XRP-USD']

In [ ]:
findata.EPOCHS=200
model = "etf-1a-test"
tickers = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]

In [ ]:
findata.EPOCHS=200
model = "mcap-1b-test"
tickers = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']

In [ ]:
pipeline.IS_VERBOSE = False
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


df  = pipeline.runModelCombinedVola(tickers, model, mod, False, loss=lossfn)


In [9]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)


,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
8,MATIC-USD,0.45,0.74,10.55,10.15,0.85,1.060000,0.24,0.61,9.34,8.23,1.170000,0.37,0.72,10.75,13.33,0.820000,-0.04,0.24,6.316667
9,SOL-USD,16.13,0.82,1237.43,1142.56,108.06,89.459999,-0.17,0.53,381.11,688.42,123.410004,0.14,0.57,755.21,702.87,90.889999,-0.16,-0.16,7.075000
5,ETH-USD,1095.74,0.85,24038.97,16952.69,2492.74,2113.270020,-0.15,0.64,15768.78,12260.69,2766.919922,0.11,0.68,14107.42,16732.71,2124.199951,-0.15,-0.15,7.500000
2,BTC-USD,17332.33,0.81,232390.92,251868.71,47248.20,39959.011719,-0.15,0.62,161940.82,140795.26,47515.781250,0.01,0.61,153777.15,127758.86,39836.558594,-0.16,-0.15,6.833333
10,TRX-USD,0.06,0.53,0.15,0.10,0.12,0.100000,-0.18,0.60,0.26,-0.06,0.110000,-0.08,0.39,0.12,-0.07,0.100000,-0.20,-0.18,2.688889
6,LINK-USD,6.74,0.79,142.44,116.11,18.42,17.469999,-0.05,0.62,101.16,78.03,22.290001,0.21,0.62,85.62,134.88,15.520000,-0.16,-0.05,3.700000
1,AVAX-USD,12.74,0.82,690.30,671.86,38.12,36.709999,-0.04,0.57,214.06,211.98,42.180000,0.11,0.62,407.36,54.21,34.389999,-0.10,-0.04,3.700000
4,DOT-USD,4.48,0.79,172.31,196.65,7.21,6.910000,-0.04,0.48,-13.47,39.76,8.430000,0.17,0.73,252.02,224.51,6.930000,-0.04,-0.04,2.650000
11,XRP-USD,0.37,0.73,7.21,5.17,0.53,0.550000,0.04,0.44,-1.55,1.39,0.650000,0.23,0.69,3.86,5.83,0.500000,-0.04,0.04,0.550000
0,ADA-USD,0.30,0.72,4.68,10.57,0.54,0.510000,-0.06,0.55,5.31,5.53,0.640000,0.18,0.69,6.76,12.97,0.540000,-0.01,-0.01,0.000000


In [ ]:
findata.EPOCHS=200
model = "vols-12b-test"
pending = ['ASTR', 'SMWB', 'APPS', 'CZOO']
next = ['TRUP']
tickers = [ 'AIRI', 'AMSC', 'ACHR', 'AUGX', 'AUR',   'ATOM',
            'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CRCT', 'CXM',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FSR', 'FTCH',
            'GOVX', 'GRAB', 'GTE', 'HIMS', 'HIVE',
            'INLX',  'INVZ', 'JOBY',  'KIND', 'KULR',
            'LAZR', 'LUMN', 'NNOX', 'MAX', 'MTTR',
            'NEPH', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SOUN', 'STEM', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
future = []

In [10]:
pending = ['PAR']
findata.EPOCHS=200
model = "ipos-2a-test"
next = ['AYX', 'FIVE']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP', 'AYX',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SNAP', 'SNOW', 'SPLK', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']

In [ ]:
findata.EPOCHS=200
model = "cetf-1b-test"
tickers = ['ARGT', 'EWD', 'FLN', 'FM',
           'ECH', 'EPHE', 'EWA', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']

In [ ]:
findata.EPOCHS=200
model = "com-2a-test"
tickers = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']

In [ ]:
lossfn = "huber_loss"
pipeline.IS_VERBOSE = False
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(),
                         pipeline.AddVWap(),
                         pipeline.AddMA(200),
                         pipeline.Adj()
                         ]))


#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='low', trading=pipeline.LowTrading)[0]
#df  = pipeline.runModelCombined(tickers, 'ipos2-3a', mod, False, loss=lossfn, output='high', trading=pipeline.HighTrading)[0]
df  = pipeline.runModelCombinedVola(tickers, model, mod, True, loss=lossfn)

In [12]:
addAllocHL(df, 0.10, 1)
df.iloc[(df.Alloc * abs(df.Gain_f)).sort_values(ascending=False).index].head(50)
#df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Error,Accu,Buy,Sell,Last,Pred,Gain,Accu_h,Buy_h,Sell_h,Pred_h,Gain_h,Accu_l,Buy_l,Sell_l,Pred_l,Gain_l,Gain_f,Alloc
39,MDB,108.17,0.94,4611.69,2865.27,500.90,383.959991,-0.23,0.73,4188.71,3022.74,555.409973,0.11,0.71,3202.89,3008.50,380.459991,-0.24,-0.23,9.139130
34,IOT,9.30,0.90,141.87,97.30,35.47,26.940001,-0.24,0.77,158.17,125.72,35.619999,0.00,0.69,90.15,26.20,25.540001,-0.28,-0.24,8.583333
59,SMCI,22.95,0.65,1752.96,608.32,740.29,464.799988,-0.37,0.65,997.20,315.82,634.150024,-0.14,0.28,161.25,-865.61,450.109985,-0.39,-0.37,5.554054
73,UBER,15.74,0.93,379.01,260.26,70.91,56.340000,-0.21,0.74,402.47,256.44,70.790001,-0.00,0.63,249.39,45.77,54.360001,-0.23,-0.21,8.966667
9,CFLT,17.54,0.89,216.71,330.48,31.72,23.090000,-0.27,0.68,229.84,358.43,29.100000,-0.08,0.73,197.08,285.10,25.440001,-0.20,-0.20,8.350000
60,SNAP,8.81,0.88,298.49,330.00,11.11,13.280000,0.19,0.61,133.39,269.96,17.129999,0.54,0.69,250.22,325.94,12.800000,0.15,0.19,8.168421
30,GOGO,1.83,0.94,132.01,128.69,10.01,8.400000,-0.16,0.84,204.18,162.04,11.660000,0.16,0.72,99.32,59.21,8.130000,-0.19,-0.16,9.025000
42,MPWR,141.46,0.90,4091.47,2629.91,752.31,603.200012,-0.20,0.78,4539.72,2082.93,661.549988,-0.12,0.62,2500.39,1228.92,622.770020,-0.17,-0.17,8.411765
11,CRWD,37.56,0.89,1846.06,1232.34,329.34,274.619995,-0.17,0.72,1453.39,1028.42,324.829987,-0.01,0.65,1326.23,1226.96,267.570007,-0.19,-0.17,8.252941
50,PANW,48.10,0.89,1691.31,621.66,376.90,276.839996,-0.27,0.75,1271.48,547.14,375.790009,-0.00,0.60,978.47,217.85,313.890015,-0.17,-0.17,8.252941


In [ ]:
pip install seaborn

In [ ]:
import seaborn as sns

Todos:



1.   Load previously saved model before training
2.   Allow eval only without test/train spliting
3.   Incremental data load
4.    Add S&P, QQQ etc to the model
5. Save model to google drive






In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !cp  /content/drive/MyDrive/colab/results/* ./results/
else:
  ! cp ~/Google\ Drive/My\ Drive/colab/results/* ./results/

Run following in local Jupyter

Run following 2 cells for colab to import libraries

Run following if the library code is modified locally

In [ ]:
!git clone https://github.com/ahsank/runml

In [ ]:
import os
import sys
path = os.path.expanduser('~/src/runml')
print(path)
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata

/Users/ahsank/src/runml


In [ ]:
import importlib
import sys
path = 'runml'
if not path in sys.path:
  sys.path.append(path)

from runml import pipeline,findata


In [ ]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accu']/stop_loss - (1-df['Accu'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
from importlib import reload
reload(findata)
reload(pipeline)

<module 'runml.pipeline' from '/Users/ahsank/src/runml/runml/pipeline.py'>

In [ ]:
import pandas as pd
df1 = pipeline.fetch_data('ABNB')
#df1.sort_index(inplace=True)
df1 = df1.round(3)

df2 = pd.read_csv('data/ABNB_2023-04-15.csv', index_col=0)
df2.index =  pd.to_datetime(df2.index)
# df2 = df2.round(3)
#data2 = data.tail(1000)
#pd.concat([df1,df2]).drop_duplicates(keep=False)
#df1.compare(df2)
#df2.index.dtype
df1

In [ ]:
display(df.sort_values('Gain', ascending=False))
display(df[["Buy", "Sell", "Total"]].sum()/len(df))


In [ ]:
results['ABNB'].pdata.__dict__.keys()

In [ ]:
findata.EPOCHS=10
tickers1 = ['ARKK', 'ARKW', 'DAPP', 'DTEC', 'EEM', 'FPX',
            'GBTC', 'GLD',
            'ICLN', 'IJR', 'IPO', 'IPOS', 'IWM',
            'JETS', 'KEMQ',
            'MGK', 'MGV', 'MOAT', 'MTUM',
            'QQQ', 'SLV','SMH', 'SMOG',
            'SPY', 'VNQ', 'VT', 'VTI', 'WDIV',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME' ]
mod = pipeline.RateReturnOnly(pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap()]))
df1 = pipeline.runModelCombined(tickers1, 'etf2b', mod, True)

In [ ]:
df1.sort_values('Gain', ascending=False)

In [ ]:
findata.EPOCHS=200
tickers3 = [
           'AHT', 'AMSC', 'ASTR', 'ATOM',
           'BKKT', 'BGFV', 'BGSF', 'CBNT', 'CLOV',
           'DNMR', 'ERJ', 'EVGO',
           'FSLY',  'FTCH', 'GOGO', 'HIVE',
           'ILAL', 'INLX', 'JMIA', 'JOBY',  'KULR', 'MTTR',
           'MYTE', 'NEPH', 'ONDS', 'MQ',
           'PETS', 'PTON', 'SFIX', 'SFT', 'STNE',
           'ULH', 'VRAR', 'WISH']
lossfn = "huber_loss"
# lossfn = "mean_squared_error"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df3, results = pipeline.runModelCombined(tickers3, 'vols', mod, False, loss=lossfn)

In [ ]:
df3

In [ ]:
import pandas as pd
seq = list()
for ticker, result in results.items():
    tempdf = result.final_df
    tempdf['predicted_rate'] = (tempdf['adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    tempdf['true_rate'] = (tempdf['true_adjclose_15']-tempdf['adjclose'])/tempdf['adjclose']
    seq.append(tempdf[['ticker', 'date', "predicted_rate", 'true_rate']])

detailstat = pd.concat(seq)



In [ ]:
normalstat = detailstat[detailstat['predicted_rate'].between(-2,2) & detailstat['true_rate'].between(-2,2)]
largestat = normalstat[~ normalstat['predicted_rate'].between(-0.25,0.25) & ~ normalstat['true_rate'].between(-0.25,0.25)]
largestat.sample(10)

In [ ]:
ax = largestat.plot.scatter(x='predicted_rate', y='true_rate',c='DarkBlue', figsize=(10,10))
# ax.set_yscale('log')
# ax.set_xscale('log')

In [ ]:
# import matplotlib.pyplot as plt

# plt.hist([normalstat['predicted_rate'],normalstat['predicted_rate']],
#          bins=100, range=(-1,1), color = ['r','g'])

corr = largestat[["predicted_rate", "true_rate"]].corr()

corr.style.background_gradient(cmap='coolwarm')

In [ ]:

sns.regplot(x=largestat["predicted_rate"], y=largestat["true_rate"])


In [ ]:
sns.heatmap(normalstat[["predicted_rate", "true_rate"]].corr(), annot = True, fmt='.2g',cmap= 'coolwarm')

In [13]:
!rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/


sending incremental file list
ipos-2a-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
ipos-2a-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
ipos-2a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 9,648,057 bytes  received 73 bytes  6,432,086.67 bytes/sec
total size is 189,533,488  speedup is 19.64
